### 지난 시간 복습

> user / group / other

> user는 UID로 판단

> group은 동일한 목적의 사용자들을 모아둠

사용자의 계정정보는
- `/etc/passwd` : UID 확인 가능
- `/etc/shadow` : UID의 비밀번호

그룹의 정보
- `/etc/group` : GID
- `/etc/gshadow` : GID의 비밀번호

user필드

`USER:x:UID:GID:GECOS:HOME:SHELL`

|필드|설명|
|--|--|
|USER|사용자의 이름을 나타냅니다. 사용자의 이름은 시스템에서 유일해야 하며 사용자로 로그인 할 때 사용합니다.|
|x (place holder)|사용자의 패스워드를 저장하는 필드입니다. 현재 이 필드는 사용되지 않으며, 현재 패스워드에 대한 정보는 `/etc/shadow`파일에 해시 패스워드형태로 저장됩니다.|
|UID|시스템에서 사용자를 식별하는 번호입니다. 0번은 root 사용자를 의미하며 1000번 미만은 시스템 사용자 계정으로 예약되어 있고 일반 사용자는 1000번부터 부여됩니다.|
|GID|시스템에서 그룹을 식별하는 번호입니다. 사용자와 마찬가지로 0번은 root 그룹이며 1000번 미만은 시스템 그룹으로 예약되어 있고 1000번 이상부터는 일반 그룹에서 부여됩니다.|
|GECOS|General Electric Comprehensive Operating이라는 형식의 주석 필드입닏. 사용자의 계정에 주석을 기재 할 수 있습니다.|
|HOME|사용자의 홈 디렉토리를 절대경로로 나타냅니다.|
|SHELL|사용자가 기본으로 사용할 쉘을 나타냅니다.|

> Q. 사용자 권한을 바꾸는 이유는?
- 원격에서 root를 접속한다는 것은 굉장히 위험한 일
- root 권한이 필요하다면 wheel 그룹에 포함시켜서 원격으로 접속할 수 있는 권한을 부여하는 것

### `su` vs `sudo`

> `su`
- switch user

> `sudo`
- super user do

### `sudo`
> `sudo [option] [user-name] command`
- `sudo` 명령은 로그아웃을 하지 않고 특정 사용자의 권한으로 명령을 수행할 수 있습니다.
- 명령을 사용할 때 사용자를 지정하지 않으면 `root` 사용자의 권한으로 명령을 수행합니다.
    - 이떄 사용하는 패스워드는 `기존에 로그인한 사용자의 패스워드`를 사용합니다.
- 한시적으로 명령을 수행하므로 수행한 후에는 기존 사용자의 권한으로 돌아오게 됩니다.

# 확장권한

> Q. 확장권한은 왜 나왔을까?
- 대부분의 경우 읽기, 쓰기, 실행과 같은 기본 권한을 설정하여 파일에 대한 접근을 제어할 수 있지만, 일부 특수한 목적으로 사용되는 파일은 기본 권한만으로 파일에 대한 접근을 제어하기 어려울 수도 있습니다.
- `파일에 대한 소유권이 없는 사용자도 일시적으로 파일에 대한 소유권을 갖게 해주는 것`도 확장권한이라고 한다.

실제로 어떻게 진행이 되는가?

1. 유저 A가 패스워드를 변경 : `usr/bin/passwd`
2. setuid 비트가 설정되어 있는가? : `/usr/bin/passwd (프로그램)`
3. 파일 실행 시 파일의 소유자 권한으로 전환하라! : `passwd 파일을 통해 사용자의 패스워드 변경`\
(=루트 권한을 가진다라는 의미)
4. `/usr/bin/passwd` -> `/etc/shadow` 파일을 수정

`which passwd` (파일 경로 확인)\
`ls -l /bin/passwd` (파일의 속성 확인 + setuid 확인)


> 파일을 실행할 때 기본원칙
- 파일을 실행한 사용자가 주체가 되어 실행된다.

### `setuid`
- 실행권한이 있는 바이너리 파일 또는 스크립트 파일에만 사용합니다.
- 파일의 실행하는 사용자가 주체가 되는 것이 아니라, 파일을 소유하고 있는 소유자가 사용자의 권한으로 프로세스를 실행하는 것

예시
- `[root@localhost ~]# grep "wheel" /etc/group` : wheel 그룹안에 유저 체크
- `usermod -G wheel {user}` : wheel 그룹에 유저 추가
- `su {user}` : 해당 유저로 로그인
- `which passwd`
- `ls -l usr/bin/passwd`
- 결과물 : `-rwsr-xr-x 1 root root 27586 apr 1 2020 /usr/bin/passwd` 와 같이 user에 s가 추가된 것을 확인할 수 있음

### `setgid`
- `setuid`처럼 바이너리 파일 또는 스크립트 파일에 설정되어 있으면 명령을 실행할 때 프로세스의 사용자 그룹이 파일의 사용자 그룹으로 실행됩니다.
- 일반적으로 디렉토리에 설정하여 사용하는 경우가 많습니다.

예시
- `groupadd {그룹명}` : 그룹추가
- `tail -5 /etc/group` : 그룹 추가 여부 확인
- `useradd {유저이름}` : 유저 추가
- `tail -2 /etc/passwd` : 유저 추가 여부 확인
- `id {유저이름}` : 유저 그룹 확인
- `usermod -aG {그룹명} {유저이름}` : 유저 -> 그룹에 추가하기
- 그룹에 유저 추가여부 확인 `tail -5 /etc/group`
- `ls -ld /{디렉토리 이름}` : 디렉토리에 소유자 확인
- `chgrp {그룹명} /{디렉토리 이름}` : 디렉토리 소유자 변경
- 이후 파일 생성 테스트 및 생성이 안된다면 디렉토리의 other 권한 확인하기
- `chmod g+s /{디렉토리 이름}` : setgid 부여하기


> `setgid`를 사용할 때는?
1. 일반적으로 바이너리 파일에서
2. 특정 데몬이 관리하는 디렉토리

> setgid에서 `s`와 `S`의 차이는?
- 실행권한 `x`가 있다면 소문자 없다면 대문자로 표기된다.

### Sticky bit
- 루트와 파일을 소유한 사용자만 파일을 삭제할 수 있도록 하는 것
    - 이유는 아무나 삭제하는 권한이 있으면 안됨.
    - 대표적인 예시 `ls -ld /tmp`
        - `drwxrwxrwt. 40 root root 4096 Oct 23 12:37 /tmp` 처럼 other에서 `t`로 표기됨.
    - 게시판을 생각해도 됨
> Q. 위에 배운 내용들을 언제 사용 하는가?
- 공유
- 멀티유저

# 접근 제어 리스트 (ACL)
- 권한을 세부적으로 사용해야 할 때 사용하는 것\
마지막 비트에 `+`가 보인다면 ACL이 적용된 파일이다.
- point는 특정 사용자만 지정

- `setfacl`
- `getfacl`

### 중요! Other에게 권한을 줄 때 주의할 점
- Other에게 `r_x`의 권한만 부여하고 `w`의 권한은 주지 않는다.
    - 그 이유는 타인이 소유자의 디렉토리를 임의로 수정하면 안되기 때문에 특정 파일에 acl을 주고 접근하도록 만든다면 r_x의 권한을 other에게 주고 접근할 수 있도록 변경한다.

# 작업 스케줄링
- 누가 작업을 예약할 것인지? (관리자 or 사용자)
- 단일성인지 주기성인지?
    - batch 작업이라고도 부름

### `at [option] time-spec`
- 단일성 작업
- 한 번 실행하고 나면 사라짐

### `crontab -e`
- `분 시 일 월 요일 명령어` 순으로 입력
    - 분 : 0~59 입력 가능
    - 시 : 0~23 입력 가능
    - 일 : 0~31 입력 가능
    - 월 : 1~12 입력 가능
    - 요일 : 0~7 입력 가능 (0은 일요일부터)
    - 명령어 : 실행할 명령어 또는 스크립트 파일의 경로를 입력할 수 있습니다.


crontab 응용문제 1

> 매 시 45분마다 clock이라는 파일을 만들고 싶다.

- `45 * * * * touch clock.txt`

crontab 응용문제 2

> 새로운 유저 (test03)을 생성하고 그 유저로 crontab을 거는것이 목표
1. date라는 명령어를 사용한 결과값을 리다이렉션을 이용하여 date.log라는 파일로 출력
2. 주기는 2분마다
3. 리다이렉션 이어쓰기


- `su -` : root로 변경 (유저 생성을 위해)
- `useradd test03` : 유저 생성
- `test03 passwd` : 비밀 번호 부여
- `su test03` : 유저 변경
- `crontab -e` : crontab 주기 예약
- `*/2 * * * * date >> /home/test03/date.log` : vi 내용 작성
- `cat /home/test03/date.log`

> 매달 10일 3시 5분
- `5 3 10 * *`

> 매달 10일 3시가 되면 5분 간격
- `*/5 3 10 * *`

# 어떤 유저로 crontab을 걸어야 하는지 중요하다. 아무 생각없이 root로 걸면 사고치기 쉽다.